In [1]:
import os
import json
with open("private.json") as jsn:
    api_key = json.load(jsn)["groq"]

llama3_30b = "llama3-70b-8192"
mixtral = "mixtral-8x7b-32768"


from groq import Groq
# https://console.groq.com/docs/text-chat

In [3]:

client = Groq(
    api_key=api_key
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

Fast language models are important for several reasons:

1. **Efficiency**: Fast language models are designed to process and generate text quickly, which is essential for many real-world applications such as real-time chat, search engines, and interactive voice response systems. Slow models can lead to poor user experience, increased latency, and decreased user engagement.
2. **Scalability**: Fast language models can handle large volumes of data and requests, making them suitable for use in large-scale systems and applications. Slow models can become a bottleneck in such systems, leading to decreased performance and increased costs.
3. **Cost-effective**: Fast language models can be more cost-effective than slower models, as they require less computational resources and can handle more requests per second. This can result in significant cost savings for businesses and organizations that rely on natural language processing (NLP) technology.
4. **Accessibility**: Fast language models can

In [7]:


chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "If I am starting a game of NetHack and I see the symbols @, d, and < close together on the screen, what does that most likely mean?",
        }
    ],
    model=llama3_30b
)

print(chat_completion.choices[0].message.content)

A new game of NetHack, how exciting!

If you see the symbols `@`, `d`, and `<` close together on the screen, it's likely that you're looking at the starting area of the dungeon. Here's what each symbol typically represents:

* `@` : This is your character, the adventurer.
* `d` : This is a downstairs (down staircase) symbol, indicating a way down to the next level of the dungeon.
* `<` : This is an upwards staircase, which leads back up to the previous level (if you're not on the top level, of course).

So, having these symbols close together suggests that you're starting near the entrance of the dungeon, with a staircase down to the next level nearby and a way back up to the surface. Not a bad starting position at all!


In [8]:
view = """
 ---------
|..f.....
|..<@...|
|.......+
|.......|
+.......|
---------
"""
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "I am starting a new game of NetHack.  Can you tell me what the following ASCII display means?" + view, 
        }
    ],
    model=llama3_30b
)

print(chat_completion.choices[0].message.content)

Exciting! You're starting a new game of NetHack!

Let me help you decipher the ASCII display:

* `.` represents an empty floor tile.
* `@` represents your character, the adventurer.
* `f` represents food (in this case, a piece of fruit).
* `+` represents a staircase (either up or down).
* `|` and `-` are just borders of the game window.

So, in this starting situation:

* You (`@`) are standing on a floor with some food (`f`) nearby.
* There's a staircase (`+`) on the level, which you can use to move up or down to other levels.
* The rest of the area is empty floor space (`.`).

Now, it's up to you to decide what to do next!


In [3]:

from groq import Groq
import os
import json

client = Groq(api_key = api_key)
MODEL = 'mixtral-8x7b-32768'


# Example dummy function hard coded to return the score of an NBA game
### So this here isn't any kind of real thing; it's a placeholder for something that fetches a real score, given a team name.
def get_game_score(team_name):
    """Get the current score for a given NBA game"""
    if "warriors" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "lakers" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "nuggets" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    elif "heat" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    else:
        return json.dumps({"team_name": team_name, "score": "unknown"})

### There's a system message that's sort of vague in terms of what happens.
### You also send tools =, which appears to be a list of JSON-like objects.  A function looks like so:
    # type = function
    # function is a dictionary with name, description, and parametes.  Name and description are straightforward.
    # I'm a little confused by the type=object.  properties may be a list of dictionaries with type and description for each argument.
    # required [] might just let you know which arguments are not optional?
def run_conversation(user_prompt):
    # Step 1: send the conversation and available functions to the model
    messages=[
        {
            "role": "system",
            "content": "You are a function calling LLM that uses the data extracted from the get_game_score function to answer questions around NBA game scores. Include the team and their opponent in your response."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_game_score",
                "description": "Get the score for a given NBA game",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "team_name": {
                            "type": "string",
                            "description": "The name of the NBA team (e.g. 'Golden State Warriors')",
                        }
                    },
                    "required": ["team_name"],
                },
            },
        }
    ]
    ### Now there's a query, with tools and messages.
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )
    print("first response")
    print(response)

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_game_score": get_game_score,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                team_name=function_args.get("team_name")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content

user_prompt = "What was the score of the Warriors game?"
print(run_conversation(user_prompt))



The score of the Golden State Warriors' game was 128 to 121, with the Warriors winning against the Los Angeles Lakers.


In [5]:
### Let's break that out of the function and see if it works.
from groq import Groq
import os
import json

client = Groq(api_key = api_key)
MODEL = 'mixtral-8x7b-32768'


# Example dummy function hard coded to return the score of an NBA game
### So this here isn't any kind of real thing; it's a placeholder for something that fetches a real score, given a team name.
def get_game_score(team_name):
    """Get the current score for a given NBA game"""
    if "warriors" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "lakers" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "nuggets" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    elif "heat" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    else:
        return json.dumps({"team_name": team_name, "score": "unknown"})

### There's a system message that's sort of vague in terms of what happens.
### You also send tools =, which appears to be a list of JSON-like objects.  A function looks like so:
    # type = function
    # function is a dictionary with name, description, and parametes.  Name and description are straightforward.
    # I'm a little confused by the type=object.  properties may be a list of dictionaries with type and description for each argument.
    # required [] might just let you know which arguments are not optional?

    # Step 1: send the conversation and available functions to the model

user_prompt = "What was the score of the Warriors game?"

messages=[
        {
            "role": "system",
            "content": "You are a function calling LLM that uses the data extracted from the get_game_score function to answer questions around NBA game scores. Include the team and their opponent in your response."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_game_score",
                "description": "Get the score for a given NBA game",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "team_name": {
                            "type": "string",
                            "description": "The name of the NBA team (e.g. 'Golden State Warriors')",
                        }
                    },
                    "required": ["team_name"],
                },
            },
        }
    ]
    ### Now there's a query, with tools and messages.
response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

    # Step 2: check if the model wanted to call a function
if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_game_score": get_game_score,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                team_name=function_args.get("team_name")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )  # get a new response from the model where it can see the function response


print(second_response.choices[0].message.content)



NameError: name 'second_response' is not defined